# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Mar 1 2023 9:13AM,258425,16,9217713,Sartorius Bioprocess Solutions,Released
1,Mar 1 2023 9:10AM,258424,10,Enova-11429,Emerginnova,Released
2,Mar 1 2023 9:10AM,258424,10,Eme-109616,Emerginnova,Released
3,Mar 1 2023 8:57AM,258422,16,9212075,Sartorius Bioprocess Solutions,Released
4,Mar 1 2023 8:37AM,258421,12,HH8676,Hush Hush,Released
5,Mar 1 2023 8:37AM,258419,12,HH-40275,Hush Hush,Released
6,Mar 1 2023 8:37AM,258419,12,HH-40276,Hush Hush,Released
7,Mar 1 2023 8:37AM,258419,12,HH-40277,Hush Hush,Released
8,Mar 1 2023 8:37AM,258419,12,HH-40278,Hush Hush,Released
9,Mar 1 2023 8:37AM,258419,12,HH-40279,Hush Hush,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
24,258420,Released,2
25,258421,Released,1
26,258422,Released,1
27,258424,Released,2
28,258425,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Cancelled,Completed,Executing,Released
id,,,,
258420,NaN,NaN,NaN,2.0
258421,NaN,NaN,NaN,1.0
258422,NaN,NaN,NaN,1.0
258424,NaN,NaN,NaN,2.0
258425,NaN,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Cancelled,Completed,Executing,Released
id,,,,
258349,1.0,2.0,10.0,2.0
258351,0.0,0.0,0.0,1.0
258353,0.0,0.0,0.0,17.0
258354,0.0,0.0,0.0,49.0
258356,0.0,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Cancelled,Completed,Executing,Released
id,,,,
258349,1.0,2,10,2
258351,0.0,0,0,1
258353,0.0,0,0,17
258354,0.0,0,0,49
258356,0.0,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Cancelled,Completed,Executing,Released
0,258349,1.0,2,10,2
1,258351,0.0,0,0,1
2,258353,0.0,0,0,17
3,258354,0.0,0,0,49
4,258356,0.0,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Cancelled,Completed,Executing,Released
0,258349,1.0,2,10,2
1,258351,0.0,,,1
2,258353,0.0,,,17
3,258354,0.0,,,49
4,258356,0.0,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id             int64
Cancelled    float64
Completed     object
Executing     object
Released      object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Mar 1 2023 9:13AM,258425,16,Sartorius Bioprocess Solutions
1,Mar 1 2023 9:10AM,258424,10,Emerginnova
3,Mar 1 2023 8:57AM,258422,16,Sartorius Bioprocess Solutions
4,Mar 1 2023 8:37AM,258421,12,Hush Hush
5,Mar 1 2023 8:37AM,258419,12,Hush Hush
48,Mar 1 2023 8:34AM,258420,19,"Zymergen, Inc."
50,Mar 1 2023 8:33AM,258418,12,Hush Hush
51,Mar 1 2023 8:30AM,258402,10,"Citieffe, Inc."
52,Mar 1 2023 8:30AM,258397,10,Bio-PRF
58,Mar 1 2023 8:13AM,258417,22,"NBTY Global, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Cancelled,Completed,Executing,Released
0,Mar 1 2023 9:13AM,258425,16,Sartorius Bioprocess Solutions,0.0,,,1
1,Mar 1 2023 9:10AM,258424,10,Emerginnova,0.0,,,2
2,Mar 1 2023 8:57AM,258422,16,Sartorius Bioprocess Solutions,0.0,,,1
3,Mar 1 2023 8:37AM,258421,12,Hush Hush,0.0,,,1
4,Mar 1 2023 8:37AM,258419,12,Hush Hush,0.0,,7,36
5,Mar 1 2023 8:34AM,258420,19,"Zymergen, Inc.",0.0,,,2
6,Mar 1 2023 8:33AM,258418,12,Hush Hush,0.0,,,1
7,Mar 1 2023 8:30AM,258402,10,"Citieffe, Inc.",0.0,,1,
8,Mar 1 2023 8:30AM,258397,10,Bio-PRF,0.0,,2,4
9,Mar 1 2023 8:13AM,258417,22,"NBTY Global, Inc.",0.0,,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Mar 1 2023 9:13AM,258425,16,Sartorius Bioprocess Solutions,1,,
1,Mar 1 2023 9:10AM,258424,10,Emerginnova,2,,
2,Mar 1 2023 8:57AM,258422,16,Sartorius Bioprocess Solutions,1,,
3,Mar 1 2023 8:37AM,258421,12,Hush Hush,1,,
4,Mar 1 2023 8:37AM,258419,12,Hush Hush,36,7,
5,Mar 1 2023 8:34AM,258420,19,"Zymergen, Inc.",2,,
6,Mar 1 2023 8:33AM,258418,12,Hush Hush,1,,
7,Mar 1 2023 8:30AM,258402,10,"Citieffe, Inc.",,1,
8,Mar 1 2023 8:30AM,258397,10,Bio-PRF,4,2,
9,Mar 1 2023 8:13AM,258417,22,"NBTY Global, Inc.",1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Mar 1 2023 9:13AM,258425,16,Sartorius Bioprocess Solutions,1,,
1,Mar 1 2023 9:10AM,258424,10,Emerginnova,2,,
2,Mar 1 2023 8:57AM,258422,16,Sartorius Bioprocess Solutions,1,,
3,Mar 1 2023 8:37AM,258421,12,Hush Hush,1,,
4,Mar 1 2023 8:37AM,258419,12,Hush Hush,36,7,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

C:\Users\jmeza.WOODFIELD\.conda\envs\PythonData\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\jmeza.WOODFIELD\.conda\envs\PythonData\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\jmeza.WOODFIELD\.conda\envs\PythonData\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Mar 1 2023 9:13AM,258425,16,Sartorius Bioprocess Solutions,1.0,NaN,NaN
1,Mar 1 2023 9:10AM,258424,10,Emerginnova,2.0,NaN,NaN
2,Mar 1 2023 8:57AM,258422,16,Sartorius Bioprocess Solutions,1.0,NaN,NaN
3,Mar 1 2023 8:37AM,258421,12,Hush Hush,1.0,NaN,NaN
4,Mar 1 2023 8:37AM,258419,12,Hush Hush,36.0,7.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Mar 1 2023 9:13AM,258425,16,Sartorius Bioprocess Solutions,1.0,0.0,0.0
1,Mar 1 2023 9:10AM,258424,10,Emerginnova,2.0,0.0,0.0
2,Mar 1 2023 8:57AM,258422,16,Sartorius Bioprocess Solutions,1.0,0.0,0.0
3,Mar 1 2023 8:37AM,258421,12,Hush Hush,1.0,0.0,0.0
4,Mar 1 2023 8:37AM,258419,12,Hush Hush,36.0,7.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,3617246,123.0,20.0,2.0
12,775258,38.0,7.0,0.0
16,516847,2.0,0.0,0.0
19,516834,17.0,10.0,0.0
22,258417,1.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,3617246,123.0,20.0,2.0
1,12,775258,38.0,7.0,0.0
2,16,516847,2.0,0.0,0.0
3,19,516834,17.0,10.0,0.0
4,22,258417,1.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,123.0,20.0,2.0
1,12,38.0,7.0,0.0
2,16,2.0,0.0,0.0
3,19,17.0,10.0,0.0
4,22,1.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,123.0
1,12,Released,38.0
2,16,Released,2.0
3,19,Released,17.0
4,22,Released,1.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,16,19,22
Status,,,,,
Completed,2.0,0.0,0.0,0.0,0.0
Executing,20.0,7.0,0.0,10.0,0.0
Released,123.0,38.0,2.0,17.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,16,19,22
0,Completed,2.0,0.0,0.0,0.0,0.0
1,Executing,20.0,7.0,0.0,10.0,0.0
2,Released,123.0,38.0,2.0,17.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,16,19,22
0,Completed,2.0,0.0,0.0,0.0,0.0
1,Executing,20.0,7.0,0.0,10.0,0.0
2,Released,123.0,38.0,2.0,17.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()